 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/json_preprocess_step1.ipynb#预处理一" data-toc-modified-id="预处理一-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>预处理一</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/json_preprocess_step1.ipynb#导入工具包" data-toc-modified-id="导入工具包-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>导入工具包</a></span></li><li><span><a href="http://localhost:8888/notebooks/json_preprocess_step1.ipynb#文件路径的确定" data-toc-modified-id="文件路径的确定-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>文件路径的确定</a></span></li><li><span><a href="http://localhost:8888/notebooks/json_preprocess_step1.ipynb#数据转换" data-toc-modified-id="数据转换-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>数据转换</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/json_preprocess_step1.ipynb#convert2coco(caption_json,-img_dir)" data-toc-modified-id="convert2coco(caption_json,-img_dir)-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>convert2coco(caption_json, img_dir)</a></span></li><li><span><a href="http://localhost:8888/notebooks/json_preprocess_step1.ipynb#convert2coco_val(caption_json,-img_dir)" data-toc-modified-id="convert2coco_val(caption_json,-img_dir)-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>convert2coco_val(caption_json, img_dir)</a></span></li><li><span><a href="http://localhost:8888/notebooks/json_preprocess_step1.ipynb#convert2coco_eval(caption_json,-img_dir)" data-toc-modified-id="convert2coco_eval(caption_json,-img_dir)-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>convert2coco_eval(caption_json, img_dir)</a></span></li><li><span><a href="http://localhost:8888/notebooks/json_preprocess_step1.ipynb#convert2coco_test(caption_json,-img_dir)" data-toc-modified-id="convert2coco_test(caption_json,-img_dir)-1.3.4"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>convert2coco_test(caption_json, img_dir)</a></span></li><li><span><a href="http://localhost:8888/notebooks/json_preprocess_step1.ipynb#数据转换" data-toc-modified-id="数据转换-1.3.5"><span class="toc-item-num">1.3.5&nbsp;&nbsp;</span>数据转换</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/json_preprocess_step1.ipynb#生成新的训练集" data-toc-modified-id="生成新的训练集-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>生成新的训练集</a></span></li></ul></li></ul></div>

# 预处理一
这是Image_Caption第一个运行的代码，核心功能就是将数据格式进行转换成和coco数据类似的格式,方便后续处理.

## 导入工具包

In [ ]:
from __future__ import print_function
import os
import argparse
import json
from PIL import Image
import jieba

## 文件路径的确定
下面这些位置对应的数据全部来自于官方给的数据.(https://challenger.ai/competition/caption/subject),可以自行下载.

- train_caption_json: 训练集对应的图片的语句解释（5句话）
- train_img_dir: 训练集中图片对应的位置
- val_caption_json: 验证集对应的图片的语句解释（5句话）
- val_img_dir：验证集中图片对应的位置
- test_img_dir: 测试集中图片对应的位置

In [ ]:
train_caption_json = '/home/jiangqy/EXP/competition/data/captions/ai_challenger_caption_train_20170902/caption_train_annotations_20170902.json'
train_img_dir = '/home/jiangqy/EXP/competition/data/captions/ai_challenger_caption_train_20170902/caption_train_images_20170902'
val_caption_json = '/home/jiangqy/EXP/competition/data/captions/ai_challenger_caption_validation_20170910/caption_validation_annotations_20170910.json'
val_img_dir = '/home/jiangqy/EXP/competition/data/captions/ai_challenger_caption_validation_20170910/caption_validation_images_20170910'
test_img_dir = '/home/jiangqy/EXP/competition/data/captions/ai_challenger_caption_test1_20170923/caption_test1_images_20170923'

## 数据转换
**四个函数解释 ** 
下面的四个函数可以认为是一个函数,都是希望将原始的官方的数据转换成coco的格式,方便后续的训练和测试以及线下的验证.

**<font colot = red>此处并不涉及任何图片特征的处理 </font> **

- convert2coco(caption_json, img_dir): 将官方给的训练数据转化为coco形式进行存储
- convert2coco_val(caption_json, img_dir): 将官方给的验证数据转化为coco形式进行存储
- convert2coco_eval(caption_json, img_dir): 将官方给的验证数据转化为coco形式进行存储(主要是为了线下进行BLEU@4的指标等计算)
- convert2coco_test(caption_json, img_dir)： 将官方给的测试集数据转化为coco形式进行存储

具体的细节可以看下面的每个函数的解释

### convert2coco(caption_json, img_dir)
将训练集的图片和文字进行一一对应并进行存储.<br />

每一个coco的形式为：<font color =red>['images','annotations']</font> 其中images存储图片的信息,包括图片的位置,图片的长宽,图片的名字以及图片的url等信息，而annotations则存储图片的id以及图片对应的描述性的话.(每张图片有5句描述的话,一张图片对应5句话一起当做一个example进行存储),所以最终coco中['images','annotations']的大小为210000(训练图片的个数).
<br />

最终的结果存储在目录:<font /color = blue>output_file = os.path.join('./json_preprocess_data', 'coco_'+os.path.basename(caption_json))</font>,可以自行修改位置.

In [ ]:
# change original form into coco form 
# save the changed form in out_put_file 
def convert2coco(caption_json, img_dir):
    dataset = json.load(open(caption_json, 'r'))
    imgdir = img_dir

    coco = dict()
    coco[u'info'] = { u'desciption':u'AI challenger image caption in mscoco format'}
    coco[u'licenses'] = ['Unknown', 'Unknown']
    coco[u'images'] = list()
    coco[u'annotations'] = list()

    for ind, sample in enumerate(dataset):
        img = Image.open(os.path.join(imgdir, sample['image_id']))
        width, height = img.size

        coco_img = {}
        coco_img[u'license'] = 0 #不用管
        coco_img[u'file_name'] = os.path.split(img_dir)[-1]+'/'+sample['image_id'] #图片的文件名称
        coco_img[u'width'] = width  #图片的宽
        coco_img[u'height'] = height #图片的高
        coco_img[u'date_captured'] = 0
        coco_img[u'coco_url'] = sample['url']  #图片对应的网址，可以输入url找到对应的图片
        coco_img[u'flickr_url'] = sample['url'] #图片对应的网址
        coco_img['id'] = os.path.splitext(os.path.basename(sample['image_id']))[0] #图片的id,我们用图片的名字作为图片的id

        coco_anno = {}
        coco_anno[u'image_id'] = os.path.splitext(os.path.basename(sample['image_id']))[0] #图片的id,我们用图片的名字作为图片的id，这个id用来对应图片与文字
        coco_anno[u'id'] = os.path.splitext(os.path.basename(sample['image_id']))[0] #图片的id,我们用图片的名字作为图片的id，这个id用来对应图片与文字
        coco_anno[u'caption'] = sample['caption'] #图片对应的文字的描述

        coco[u'images'].append(coco_img) 
        coco[u'annotations'].append(coco_anno)
        if ind % 1000 == 0:
            print('{}/{}'.format(ind, len(dataset)))

    output_file = os.path.join('./json_preprocess_data', 'coco_'+os.path.basename(caption_json))  #存储
    with open(output_file, 'w') as fid:
        json.dump(coco, fid)
    print('Saved to {}'.format(output_file))
    return coco

### convert2coco_val(caption_json, img_dir)
将验证集的图片和文字进行一一对应并进行存储.<br />

每一个coco的形式和训练集类似：<font color =red>['images','annotations']</font> 其中images存储图片的信息,包括图片的位置,图片的长宽,图片的名字以及图片的url等信息，而annotations则存储图片的id以及图片对应的描述性的话.(每张图片有5句描述的话),所以最终coco的大小为30000,其中30000为验证图片的个数.和训练集是类似的
<br />

**注意** <br />
中间多了这样一段代码:(总体对后续的处理不影响,因为验证集中每张图片基本也都是5句话,可以认为就是和train一样的处理)
<font color=red>该部分很少被运行,主要解决上面那个图中的情况，有些captions中间断开了，为空，那我们就用它上面一句话来补充它形成5句话 </font>
<font color =blue>
          $  for s in sample['caption']:   
            if len(s)==0:   
                print(ind,'***********')  
                print('error: some caption had no words?')   
                print(coco_img[u'file_name'])   
                sample['caption'][idx] = sample['caption'][idx-1]   
                print(sample['caption'])  
                print(len(sample['caption']),len(coco_anno[u'caption']))  
                print(ind,'***********')  
                #break   
            idx = idx+1   
        coco[u'images'].append(coco_img) 
        coco[u'annotations'].append(coco_anno)  $
</font><br / >
        
            

最终的结果存储在目录:<font /color = blue>output_file = os.path.join('./json_preprocess_data', 'coco_'+os.path.basename(caption_json))</font>,可以自行修改位置.

![](./pictures_for_jupyter/example.png)

In [1]:
def convert2coco_val(caption_json, img_dir):
    dataset = json.load(open(caption_json, 'r'))
    imgdir = img_dir

    coco = dict()
    coco[u'info'] = { u'desciption':u'AI challenger image caption in mscoco format'}
    coco[u'licenses'] = ['Unknown', 'Unknown']
    coco[u'images'] = list()
    coco[u'annotations'] = list()

    for ind, sample in enumerate(dataset):
        img = Image.open(os.path.join(imgdir, sample['image_id']))
        width, height = img.size

        coco_img = {}
        coco_img[u'license'] = 0
        coco_img[u'file_name'] = os.path.split(img_dir)[-1]+'/'+sample['image_id']
        coco_img[u'width'] = width
        coco_img[u'height'] = height
        coco_img[u'date_captured'] = 0
        coco_img[u'coco_url'] = sample['url']
        coco_img[u'flickr_url'] = sample['url']
        coco_img['id'] = os.path.splitext(os.path.basename(sample['image_id']))[0]

        coco_anno = {}
        coco_anno[u'image_id'] = os.path.splitext(os.path.basename(sample['image_id']))[0]
        coco_anno[u'id'] = os.path.splitext(os.path.basename(sample['image_id']))[0]
        coco_anno[u'caption'] = sample['caption']
        idx = 0
        for s in sample['caption']:  #该部分很少被运行,主要解决上面那个图中的情况，有些captions中间断开了，为空，那我们就用它上面一句话来补充它形成5句话
            if len(s)==0: 
                print('error: some caption had no words?')
                print(coco_img[u'file_name'])
                sample['caption'][idx] = sample['caption'][idx-1]
                print(sample['caption'])
                print(len(sample['caption']),len(coco_anno[u'caption'])) 
                #break
            idx = idx+1
        coco[u'images'].append(coco_img)
        coco[u'annotations'].append(coco_anno)
        if ind % 1000 == 0:
            print('{}/{}'.format(ind, len(dataset)))

    output_file = os.path.join('./json_preprocess_data', 'coco_'+os.path.basename(caption_json))
    with open(output_file, 'w') as fid:
        json.dump(coco, fid)
    print('Saved to {}'.format(output_file))
    return coco

### convert2coco_eval(caption_json, img_dir)
该函数主要是为了方便计算线下的一些指标,例如BLEU@4这些指标等,类似于线上的评估.如果不打算进行线下的评估可以不适用该函数.<br />

该函数有与convert2coco_val有几个区别,将图片的长宽都设置为了224 $*$ 224的，大体的思路和上面一样,是为了后续线下进行验证准备的数据集,所以前期的训练暂时会用不到. 

每一个coco的形式和训练集类似：<font color =red>['images','annotations']</font> 其中images存储图片的信息,包括图片的位置,图片的长宽,图片的名字以及图片的url等信息，而annotations则存储图片的id以及图片对应的描述性的话.(每张图片有5句描述的话,这边后面会有5张一样的图片对应5句话),所以最终coco的大小为30000 * 5,其中30000为验证图片的个数.
<br />

**注意** <br />
中间多了这样一段代码:(总体对后续的处理不影响,可以认为就是和train一样的处理)
<font color =blue>  

            for coco_anno_ in coco_anno['caption']:   
            coco_anno_s = {}  
            coco_anno_s[u'image_id'] = coco_anno[u'image_id']  
            coco_anno_s[u'id'] = coco_anno[u'id']  
            w = jieba.cut(coco_anno_.strip(), cut_all=False) 
            p =" ".join(w) 
            coco_anno_ = p 
            coco_anno_s[u'caption'] = coco_anno_ 
            coco[u'annotations'].append(coco_anno_s)
</font>    
最终的结果存储在目录:<font /color = blue>output_file = os.path.join('./json_preprocess_data', 'coco_'+os.path.basename(caption_json))</font>,可以自行修改位置.

In [ ]:
def convert2coco_eval(caption_json, img_dir):
    dataset = json.load(open(caption_json, 'r'))
    imgdir = img_dir

    coco = dict()
    coco[u'info'] = { u'desciption':u'AI challenger image caption in mscoco format'}
    coco[u'licenses'] = ['Unknown', 'Unknown']
    coco[u'images'] = list()
    coco[u'annotations'] = list()
    coco[u'type'] = u'captions'
    for ind, sample in enumerate(dataset):
        #img = Image.open(os.path.join(imgdir, sample['image_id']))
        #width, height = img.size
        width, height = 224, 224

        coco_img = {}
        coco_img[u'license'] = 0
        coco_img[u'file_name'] = os.path.split(img_dir)[-1]+'/'+sample['image_id']
        coco_img[u'width'] = width
        coco_img[u'height'] = height
        coco_img[u'date_captured'] = 0
        coco_img[u'coco_url'] = sample['url']
        coco_img[u'flickr_url'] = sample['url']
        coco_img['id'] = os.path.splitext(os.path.basename(sample['image_id']))[0]

        coco_anno = {}
        coco_anno[u'image_id'] = os.path.splitext(os.path.basename(sample['image_id']))[0]
        coco_anno[u'id'] = os.path.splitext(os.path.basename(sample['image_id']))[0]
        coco_anno[u'caption'] = sample['caption']

        coco[u'images'].append(coco_img)
        
        for coco_anno_ in coco_anno['caption']:
            coco_anno_s = {}
            coco_anno_s[u'image_id'] = coco_anno[u'image_id']
            coco_anno_s[u'id'] = coco_anno[u'id']
            w = jieba.cut(coco_anno_.strip(), cut_all=False)
            p = ' '.join(w)
            coco_anno_ = p
            coco_anno_s[u'caption'] = coco_anno_
            coco[u'annotations'].append(coco_anno_s)
        if ind % 1000 == 0:
            print('{}/{}'.format(ind, len(dataset)))

    output_file = os.path.join('./json_preprocess_data', 'coco_val_'+os.path.basename(caption_json))
    with open(output_file, 'w') as fid:
        json.dump(coco, fid)
    print('Saved to {}'.format(output_file))
    return coco

### convert2coco_test(caption_json, img_dir)
将测试集的图片的信息进行存储.

因为测试集中的图片是没有语句描述的,需要我们进行预测才能得到.所以和训练集以及验证集进行比较会缺少文字描述的内容,只含有<font color =red>['images']</font>内容.
 
最终的结果存储在目录:<font color = red>output_file = os.path.join('./json_preprocess_data', 'ai_challenger_test1.json')</font>,可以自行修改位置.

In [ ]:
def convert2coco_test(img_dir):
    coco = dict()
    coco[u'info'] = { u'desciption':u'AI challenger image caption in mscoco format'}
    coco[u'licenses'] = ['Unknown', 'Unknown']
    coco[u'images'] = list()
    ind = 0
    for im_name in enumerate(os.listdir(img_dir)):
        width, height = 224, 224

        coco_img = {}
        coco_img[u'license'] = 0
        coco_img[u'file_name'] = im_name[1]
        coco_img[u'width'] = width
        coco_img[u'height'] = height
        coco_img[u'date_captured'] = 0
        coco_img[u'id'] = os.path.splitext(os.path.basename(im_name[1]))[0]
        ind = ind + 1
        coco[u'images'].append(coco_img)

        print('{}/{}'.format(ind, len(os.listdir(img_dir))))

    output_file = os.path.join('./json_preprocess_data', 'ai_challenger_test1.json')
    with open(output_file, 'w') as fid:
        json.dump(coco, fid)
    print('Saved to {}'.format(output_file)) 
    return coco

### 数据转换
有了上面的四个函数,我们将我们的数据直接转换为对应的coco形式即可.

In [ ]:
convert2coco(train_caption_json, train_img_dir)
convert2coco_val(val_caption_json, val_img_dir) 
convert2coco_test(test_img_dir) 
convert2coco_eval(val_caption_json, val_img_dir)

## 生成新的训练集
这边我们将训练集和验证集合并放到一起，并用split = 'train' | 'val' 来标记该数据属于训练集还是验证集,此外我们将每个图片对应的5句话进行分词,并把所有的分词结果使用tokens进行存储.最终将结果存在:

**<font color =red> output_file = os.path.join('./json_preprocess_data', 'coco_ai_challenger_version1.json')</font>** 下面.

In [ ]:
import json
import os
import jieba
train = json.load(open('./json_preprocess_data/coco_caption_train_annotations_20170902.json', 'r'))
val = json.load(open('./json_preprocess_data/coco_caption_validation_annotations_20170910.json', 'r'))
print(val.keys())
print(val['info'])
print(len(val['images']))
print(len(val['annotations']))
print(val['images'][0])
print(val['annotations'][0])

imgs = train['images']+val['images']
annots = train['annotations']+val['annotations']

itoa = {}
for a in annots:
    imgid = a['image_id']
    if not imgid in itoa: itoa[imgid] = []
    itoa[imgid].append(a) 
    
out_json={}
out=[]
for i, img in enumerate(imgs):
    out_im = {}
    # coco specific here, they store train/val images separately
    split = 'train' if 'train' in img['file_name'] else 'val'
    annotsi = itoa[img['id']]
    sentid = 0
    out_im['cocoid'] = img['id']
    out_im['filename'] = os.path.basename(img['file_name'])
    if 'val' in img['file_name']:
        out_im['filepath'] = 'ai_challenger_caption_validation_20170910/caption_validation_images_20170910' #验证集图片存放的位置
    else:
        out_im['filepath'] = 'ai_challenger_caption_train_20170902/caption_train_images_20170902' #训练集图片存放的位置
    out_s = [] 
    for a in annotsi: 
        txt = []
        jimg = {}
        jimg['imgid'] = img['id']
        jimg['raw'] = a   # 原先的5个句子
        jimg['sentid'] = img['id']+'_'+str(sentid) 
        for s in a['caption']:
            txt.append(list(jieba.cut(s)))  
        jimg['tokens'] = txt #现在5个的句子的token组成的词拼接在一起
        jimg['sentids'] = [] 
        out_s.append(jimg) 
    out_im['sentences'] = out_s
    out_im['split'] = split
    out.append(out_im)
out_json['images']=out
out_json['dataset']='ai_challenger'
output_file = os.path.join('./json_preprocess_data', 'coco_ai_challenger_new_version.json')
json.dump(out_json, open(output_file, 'w'))